In [9]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')

# Chargement du dataset
data = pd.read_csv('data.csv')

# Suppression des lignes sans synopsis ou genres
data.dropna(subset=['overview', 'genres'], inplace=True)
data.reset_index(drop=True, inplace=True)

# Filtrage des données (on garde seulement les colonnes qui nous intéressent)
sorted_data = data[['original_title', 'overview', 'genres']]
filtered_data = sorted_data[sorted_data['overview'].str.len() > 10]

# Tokenisation et lemmatisation avec NLTK
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    tokens = word_tokenize(text)
    lemmatized_tokens = [lemmatizer.lemmatize(token.lower()) for token in tokens]
    return lemmatized_tokens

# Pré-traitement des synopsis
processed_overview = filtered_data['overview'].apply(preprocess_text)
filtered_data = filtered_data.assign(processed_overview=processed_overview)
filtered_data = filtered_data.dropna(subset=['overview'])

# Entraînement des embeddings de mots
model = Word2Vec(sentences=filtered_data['processed_overview'], vector_size=100, window=5, min_count=1, workers=4)

# Entrainement modèle de classification pour prédire les genres
X = np.array([np.mean([model.wv[word] for word in doc if word in model.wv] or [np.zeros(model.vector_size)], axis=0) for doc in filtered_data['processed_overview']])
y = filtered_data['genres']

clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X, y)

# Fonction pour recommander des films similaires en fonction du genre prédit
def similar_movies(title):
    
    # Prédire le genre du film fourni par l'utilisateur
    user_movie = data[data['original_title'].str.lower() == title.lower()]
    user_movie_description = preprocess_text(user_movie['overview'].iloc[0])
    user_movie_embedding = np.mean([model.wv[word] for word in user_movie_description if word in model.wv] or [np.zeros(model.vector_size)], axis=0)
    user_movie_genre = clf.predict([user_movie_embedding])[0]

    # Sélectionner les films du même genre
    same_genre_movies = data[data['genres'].str.contains(user_movie_genre)]

    # Calculer la similarité entre le film donné et les films du même genre
    similarities = []
    for index, movie in same_genre_movies.iterrows():
        movie_description = preprocess_text(movie['overview'])
        movie_embedding = np.mean([model.wv[word] for word in movie_description if word in model.wv] or [np.zeros(model.vector_size)], axis=0)
        similarity = cosine_similarity([user_movie_embedding], [movie_embedding])[0][0]
        similarities.append((movie['original_title'], similarity))
        print(f"Traitement du film : {movie['original_title']} - Similarité calculée : {similarity:.4f}")

    # Trier les films par similarité et renvoyer le classement
    movies = sorted(similarities, key=lambda x: x[1], reverse=True)
    return movies

title = 'Avatar'  # Exemple de titre de film utilisateur
recommendations = similar_movies(title)
print(f"===============================")
print(f"Recommandations pour '{title}':")
print(f"===============================")
for idx, (movie_title, similarity) in enumerate(recommendations[:10]):
    print(f"{idx + 1}. {movie_title} (Similarité : {similarity:.4f})")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Benjamin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Traitement du film : Avatar - Similarité calculée : 1.0000
Traitement du film : Pirates of the Caribbean: At World's End - Similarité calculée : 0.9905
Traitement du film : Spectre - Similarité calculée : 0.9978
Traitement du film : The Dark Knight Rises - Similarité calculée : 0.9997
Traitement du film : John Carter - Similarité calculée : 0.9984
Traitement du film : Spider-Man 3 - Similarité calculée : 0.9987
Traitement du film : Tangled - Similarité calculée : 0.9990
Traitement du film : Avengers: Age of Ultron - Similarité calculée : 0.9972
Traitement du film : Harry Potter and the Half-Blood Prince - Similarité calculée : 0.9983
Traitement du film : Batman v Superman: Dawn of Justice - Similarité calculée : 0.9975
Traitement du film : Superman Returns - Similarité calculée : 0.9967
Traitement du film : Quantum of Solace - Similarité calculée : 0.9978
Traitement du film : Pirates of the Caribbean: Dead Man's Chest - Similarité calculée : 0.9941
Traitement du film : The Lone Ranger 